# Model serialization

```{autolink-concat}
```

This page demonstrates a strategy for exporting an amplitude model with its suggested parameter defaults to disk and loading it back into memory later on for computations with the computational backend.

In [ ]:
from __future__ import annotations

import os
import pickle

import cloudpickle
import jax.numpy as jnp
import sympy as sp
from tensorwaves.function.sympy import create_function

from polarimetry.io import mute_jax_warnings, perform_cached_doit
from polarimetry.lhcb import load_model
from polarimetry.lhcb.particle import load_particles

mute_jax_warnings()

## Export model

In [ ]:
data_dir = "../../data"
particles = load_particles(f"{data_dir}/particle-definitions.yaml")
model = load_model(f"{data_dir}/model-definitions.yaml", particles, model_id=0)
unfolded_intensity_expr = perform_cached_doit(model.full_expression)

In [ ]:
free_parameters = {
    symbol: value
    for symbol, value in model.parameter_defaults.items()
    if isinstance(symbol, sp.Indexed)
    if "production" in str(symbol)
}
fixed_parameters = {
    symbol: value
    for symbol, value in model.parameter_defaults.items()
    if symbol not in free_parameters
}
subs_intensity_expr = unfolded_intensity_expr.xreplace(fixed_parameters)

In [ ]:
def sigma3via12() -> dict[sp.Symbol, sp.Expr]:
    s1, s2, s3 = sp.symbols("sigma1:4", nonnegative=True)
    m0, m1, m2, m3 = sp.symbols("m:4", nonnegative=True)
    return {s3: m1**2 + m2**2 + m3**2 + m0**2 - s1 - s2}

In [ ]:
dict_forms = {
    "intensity_expr": unfolded_intensity_expr,
    "variables": {k: v.doit() for (k, v) in model.variables.items()},
    "parameter_defaults": model.parameter_defaults,
    "sigma3": sigma3via12(),
}

In [ ]:
filename = "exported-model.pkl"
with open(filename, "wb") as f:
    cloudpickle.dump(dict_forms, f)

## Import model

The model is saved in a Python {obj}`dict` and to a {mod}`pickle` file. The dictionary contains a SymPy expressions for the model and suggested parameter default values. These parameter and variable symbols are substituted using the `fully_substitute()` function.

In [ ]:
def load_model(filename: str) -> dict:
    if not os.path.exists(filename):
        raise ValueError(f"The input file not found at ${filename}")
    with open(filename, "rb") as f:
        return pickle.load(f)

In [ ]:
def fully_substitute(model_description: dict) -> sp.Expr:
    expr = (
        model_description["intensity_expr"]
        .xreplace(model_description["variables"])
        .xreplace(model_description["sigma3"])
        .xreplace(model_description["parameter_defaults"])
    )
    return expr

In [ ]:
imported_model = load_model(filename)
intensity_on_2vars = fully_substitute(imported_model)

### Compilation

The resulting symbolic expression depends on two variables:

- $\sigma_1 = m_{K\pi}^2$, mass of the $K^- \pi^+$ system, and
- $\sigma_2 = m_{pK}^2$, mass of the $p K^-$ system.

This expression is turned into a numerical function by either {func}`~sympy.utilities.lambdify.lambdify`, using {doc}`JAX<jax:index>` as a computational backend.

For {mod}`sympy` backend the position argument are used.

In [ ]:
s12 = sp.symbols("sigma1:3", nonnegative=True)
assert intensity_on_2vars.free_symbols == set(s12)

func = sp.lambdify(s12, intensity_on_2vars)

In [ ]:
func(1.0, 3.0), func(1.1, 3.2)

The compilation to JAX is facilitated by {mod}`tensorwaves`:

In [ ]:
density = create_function(intensity_on_2vars, backend="jax")

In [ ]:
density({"sigma1": jnp.array([1.0, 1.1]), "sigma2": jnp.array([3.0, 3.2])})